# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [20]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [21]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [22]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [23]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [24]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [25]:
session.execute('''
                CREATE KEYSPACE IF NOT EXISTS datasparkify
                WITH REPLICATION={'class': 'SimpleStrategy', 'replication_factor': 1}
                ''')

#### Set Keyspace

In [26]:
session.set_keyspace('datasparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
# choose (sessionId, itemInSession) as the unique primary key
# for the select query, choose the data where the values of (sessionId, itemInSession) from the question

In [35]:
session.execute('''
                CREATE TABLE IF NOT EXISTS song_list(
                artist TEXT,
                songlength FLOAT,
                songtitle TEXT,
                sessionId INT,
                itemInSession INT,
                PRIMARY KEY(sessionId, itemInSession)
                )''')
                    

In [36]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_list(artist, songlength, songtitle, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], float(line[5]), line[9], int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [37]:
question_one_query='''
SELECT artist, songtitle, songlength from datasong
where sessionId=338 and itemInSession=4'''

song_list_query_data=session.execute(question_one_query)
song_list_query_data_df=pd.DataFrame(song_list_query_data._current_rows)
song_list_query_data_df

,artist,songtitle,songlength
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
# Use clustering key, userID and sessionID as partition keys. then add item_in_session because we want to sort our data based on itemInsession.
# for the select query, just choose the data question asks and add the condition: userId=10 and sessionId=182

In [33]:
session.execute('''
                CREATE TABLE IF NOT EXISTS song_playlist(
                userId INT,
                sessionId INT,
                item_insession INT,
                artist TEXT,
                songtitle TEXT,
                firstname TEXT,
                lastname TEXT,
                PRIMARY KEY ((userId, sessionId), item_insession)
                )''')


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_playlist(userId, sessionId, item_insession, artist, songtitle, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

question_two_query='''
SELECT artist, songtitle, firstname, lastname from song_playlist
where userId=10 and sessionId=182'''

song_playlist_query_data=session.execute(question_two_query)
song_playlist_query_data_df=pd.DataFrame(song_playlist_query_data._current_rows)
song_playlist_query_data_df       
                    

,artist,songtitle,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [ ]:
# choose (songtitle, userId) as the primary key since same userid may have multiple songs
# for the select query, choose the data that question asks and add the condition songtitle name

In [34]:
session.execute('''
                CREATE TABLE IF NOT EXISTS song_user_list(
                songtitle TEXT,
                userId INT,
                firstname TEXT,
                lastname TEXT,
                PRIMARY KEY (songtitle, userId)
                )''')


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_user_list(songtitle, userId, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

question_three_query="""
SELECT firstname, lastname from song_user_list
WHERE songtitle='All Hands Against His Own'
"""

song_user_list_query_data=session.execute(question_three_query)
song_user_list_query_data_df=pd.DataFrame(song_user_list_query_data._current_rows)
song_user_list_query_data_df  
                    

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [38]:
session.execute("DROP TABLE IF EXISTS song_list")
session.execute("DROP TABLE IF EXISTS song_playlist")
session.execute("DROP TABLE IF EXISTS song_user_list")

### Close the session and cluster connection¶

In [39]:
session.shutdown()
cluster.shutdown()